Tensorlines are computed using the classic RK4-integration scheme. At every intermediate update-step the eigenvector-field needs to be locally re-oriented. In order to avoid running into tensorline singularites (=points where the eigenvectors are not uniquely defined), we rescale the eigenvector field such that tensorline singularities are turned into fixed points.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid |
| Y | array (Ny, Nx) | Y-meshgrid |
| defined domain | boolean array (Ny, Nx) | grid specifying whether velocity field is defined (=1) or undefined (=0)|
| x | array (2, Npoints) | position (#Npoints = Number of initial conditions) |
| x_prime | array (2, Npoints) | eigenvector at point 'x' (=tangent to tensorline)|
| step_size | float | fixed step-size for integration of tensorlines|
| vector_field | array(Ny, Nx) | eigenvector field (not oriented)|
| Interp_eig | object | Interpolant of eigenvalue|
| x_update | array (2, Npoints) | updated coordinate of tensorline |
| x_prime_update | array (2, Npoints) | eigenvector at 'x_update' with the same orientation as 'x_prime'|

In [ ]:
# import function which checks whether the tensorlines are still in a defined domain
from ipynb.fs.defs.check_location import check_location

# import function which does the rescaling of the vectorfield
from ipynb.fs.defs.scaling_vectorfield import _scaling_vectorfield

# import numpy
import numpy as np

In [2]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

In [1]:
def _RK4_tensorlines(X, Y, defined_domain, x, x_prime, step_size, vector_field, interp_eig):
    '''
    Computes tensorlines using RK4-integration scheme.
    
    Parameters:
        X:              array (Ny, Nx), X-meshgrid
        Y:              array (Ny, Nx), Y-meshgrid
        defined_domain: array (Ny, Nx), grid specifying whether velocity field is defined (=1) or undefined (=0)
        x:              array (2, Npoints), position (#Npoints = Number of initial conditions)
        x_prime:        array (2, Npoints), eigenvector at 'x'
        step_size:      float, step size used for integration. This value is kept constant.
        vector_field:   array (Ny, Nx, 2), eigenvector field over domain domain
        interp_eig:     Interpolant-object for eigenvalue field
        
    Returns:
        x_update:       array (2, Npoints), updated coordinate of tensorline
        x_prime:        array (2, Npoints), eigenvector at 'x_update' with the same orientation as 'x_prime'
    '''
        
    # Define starting point.
    x1 = x
        
    # Check if particle is in domain of data
    loc = check_location(X, Y, defined_domain, x1)[0]
    
    # If not in domain --> stop integration
    if loc != "IN":
        return None, None
        
    # Compute x_prime at the beginning of the time-step by re-orienting and rescaling the vector field
    x_prime = _scaling_vectorfield(X, Y, x1, x_prime, vector_field, interp_eig)
    
    # x_prime can be None at the boundaries of the spatial domain of the data --> stop integration
    if x_prime is None:
        return None, None
    
    # compute derivative
    k1 = step_size * x_prime

    #  position and time at the first midpoint.
    x2 = x1 + .5 * k1
    
    # Check if particle is in domain of data
    loc = check_location(X, Y, defined_domain, x2)[0]
    
    # If not in domain --> stop integration
    if loc != "IN":
        return None, None
        
    # Compute x_prime at the first midpoint.
    x_prime = _scaling_vectorfield(X, Y, x2, x_prime, vector_field, interp_eig)
    
    # x_prime can be None at the boundaries of the spatial domain of the data --> stop integration
    if x_prime is None:
        return None, None
    
    # compute derivative
    k2 = step_size * x_prime

    # Update position at the second midpoint.
    x3 = x1 + .5 * k2
    
    # Check if particle is in domain of data
    loc = check_location(X, Y, defined_domain, x3)[0]
    
    # If not in domain --> stop integration
    if loc != "IN": 
        return None, None
    
    # Compute x_prime at the second midpoint.
    x_prime = _scaling_vectorfield(X, Y, x3, x_prime, vector_field, interp_eig)
    
    # x_prime can be None at the boundaries of the spatial domain of the data --> stop integration
    if x_prime is None:
        return None, None
    
    # compute derivative
    k3 = step_size * x_prime
    
    # Update position at the endpoint.
    x4 = x1 + k3
    
    loc = check_location(X, Y, defined_domain, x4)[0]
    if loc != "IN":
        return None, None
    
    # Compute derivative at the end of the time-step.
    x_prime = _scaling_vectorfield(X, Y, x4, x_prime, vector_field, interp_eig) 
    
    # x_prime can be None at the boundaries of the spatial domain of the data --> stop integration
    if x_prime is None:
        return None, None
    
    # compute derivative
    k4 = step_size * x_prime
    
    # define list for derivatives and positions of particle
    x_prime_update = []
    x_update = []
        
    # Compute RK4-derivative
    for j in range(2):
        x_prime_update.append(1.0 / 6.0*(k1[j] + 2 * k2[j] + 2 * k3[j] + k4[j])/step_size)
    
    # Integration x <-- x + x_prime*step_size
    for j in range(2):
        # Update position of particles
        x_update.append(x[j] + x_prime_update[j]*step_size)

    # transform list to arrays
    x_update = np.array(x_update)
    x_prime_update = np.array(x_prime_update)
    
    # Check if particle is in domain of data
    if check_location(X, Y, defined_domain, x_update)[0] != "IN": 
        return None, None
    
    return x_update, x_prime_update